In [16]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
import dash
import dash_bootstrap_components as dbc
from datetime import datetime
from dash import html
from dash import dcc
import plotly.express as px
import calendar
from dash.dependencies import Input,Output,State

## Initializing my app

In [17]:
# Create a dash application
app = dash.Dash(__name__)
#JupyterDash.infer_jupyter_proxy_config()

# REVIEW1: Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

## Reading data 

In [18]:
#Read Data
df = pd.read_csv('df_worldwide.csv')

In [19]:
df.info
df.shape
df.isna().sum()

date                    0
Country/Region          0
Latitude                0
Longitude               0
Confirmed               0
Deaths                  0
Recovered               0
Active                  0
share_of_last_week      0
percentage              0
population            591
dtype: int64

## preprocessing and preparing Data 

In [20]:
df['date'] = pd.to_datetime(df['date'])
df.head(10)

,date,Country/Region,Latitude,Longitude,Confirmed,Deaths,Recovered,Active,share_of_last_week,percentage,population
0,2020-01-22,Afghanistan,33.939110,67.709953,0,0,0,0,0.0,0.0,389.28341
1,2020-01-22,Albania,41.153300,20.168300,0,0,0,0,0.0,0.0,28.77800
2,2020-01-22,Algeria,28.033900,1.659600,0,0,0,0,0.0,0.0,438.51043
3,2020-01-22,Andorra,42.506300,1.521800,0,0,0,0,0.0,0.0,0.77265
4,2020-01-22,Angola,-11.202700,17.873900,0,0,0,0,0.0,0.0,328.66267
5,2020-01-22,Antigua and Barbuda,17.060800,-61.796400,0,0,0,0,0.0,0.0,0.97928
6,2020-01-22,Argentina,-38.416100,-63.616700,0,0,0,0,0.0,0.0,451.95777
7,2020-01-22,Armenia,40.069100,45.038200,0,0,0,0,0.0,0.0,29.63234
8,2020-01-22,Australia,-32.106275,141.355487,0,0,0,0,0.0,0.0,254.99881
9,2020-01-22,Austria,47.516200,14.550100,0,0,0,0,0.0,0.0,90.06400


In [21]:
update = df['date'].dt.strftime('%b %d, %Y').iloc[-1]


In [22]:
value = df[df['date'] == df['date'].iloc[-1]]['Confirmed'].sum()
delta = df[df['date'] == df['date'].unique()[-2]]['Confirmed'].sum()

Deaths_value = df[df['date'] == df['date'].iloc[-1]]['Deaths'].sum()
Deaths_delta = df[df['date'] == df['date'].unique()[-2]]['Deaths'].sum()

Recovered_value = df[df['date'] == df['date'].iloc[-1]]['Recovered'].sum()
Recovered_delta = df[df['date'] == df['date'].unique()[-2]]['Recovered'].sum()

Active_value = df[df['date'] == df['date'].iloc[-1]]['Active'].sum()
Active_delta = df[df['date'] == df['date'].unique()[-2]]['Active'].sum()


In [23]:
df['month'] = df.date.dt.month
MonthsDF = df.groupby(['month']).sum()[['Confirmed','Deaths','Recovered','Active']].reset_index()
country = 'China'
countrysDF = df.groupby(['Country/Region']).sum()[['Confirmed','Deaths','Recovered','Active']].reset_index()

countryDF = countrysDF[countrysDF['Country/Region'] == country] 
countryValues = countryDF[['Confirmed','Deaths','Recovered','Active']]
x = ['Confirmed','Deaths' , 'Recovered','Active']
y = countryValues.values[0]


In [24]:
countrysDF.sort_values(by = 'Confirmed' , ascending = False , inplace = True)
top5Months =  MonthsDF.sort_values(by = 'Recovered' , ascending = False)
collectedDF = df.groupby(['Country/Region','month']).sum().reset_index()
collectedDF

,Country/Region,month,Latitude,Longitude,Confirmed,Deaths,Recovered,Active,share_of_last_week,percentage,population
0,Afghanistan,1,339.391100,677.099530,0,0,0,0,0.000000,0.0,3892.83410
1,Afghanistan,2,984.234190,1963.588637,6,0,0,6,600.000000,600.0,11289.21889
2,Afghanistan,3,1052.112410,2099.008543,1219,29,26,1164,1857.788387,1858.0,12067.78571
3,Afghanistan,4,1018.173300,2031.298590,27237,860,2927,23450,1352.664030,1352.9,11678.50230
4,Afghanistan,5,1052.112410,2099.008543,225655,4994,24129,196532,1143.112236,1142.9,12067.78571
...,...,...,...,...,...,...,...,...,...,...,...
1499,Zimbabwe,4,-570.463140,874.645710,618,86,42,490,977.458435,977.7,4458.87810
1500,Zimbabwe,5,-589.478578,903.800567,1871,124,470,1277,677.601477,677.7,4607.50737
1501,Zimbabwe,6,-570.463140,874.645710,12096,142,2113,9841,860.716259,860.6,4458.87810
1502,Zimbabwe,7,-589.478578,903.800567,48113,721,13001,34391,923.355760,923.4,4607.50737


In [25]:
month = 5
countryValues = collectedDF[(collectedDF['Country/Region'] == country) & (collectedDF.month == month)]
countryValues = countryValues[['Confirmed','Deaths','Recovered','Active']]
x = ['Confirmed','Deaths' , 'Recovered','Active']
y = countryValues.values[0]

## Dash Components Creation 

In [26]:

row = html.Div(
    [
        dbc.Row(
            dbc.Col(
                html.H1(" Covid-19 Tracking "),
                style ={'margin' : '1rem','color':'white' ,'textAlign':'center'},
                
            )
        ),
        dbc.Row(
            [
                dbc.Card(
    [
      dbc.CardHeader("Last Updated on"),
        dbc.CardBody(
            [
                html.Br(),
                html.H4(update),
                html.H5('end of day')

            ]
        )
    ],
    style={"width": "18rem", 'margin' : '1rem' , 'text-align': 'center' },
),
                dbc.Card(
    [
      dbc.CardHeader(" Cumlative Confirmed Cases "),  
        dbc.CardBody(
            [
                    html.Div(dcc.Graph(id='confirmed',
                       figure={'data': [{'type': 'indicator',
                    'mode': 'number+delta',
                    'value': value,
                    'delta': {'reference': delta,
                              'valueformat': ',g',
                              'relative': False,
                              'increasing': {'color': 'black'},
                              'decreasing': {'color': 'black'},
                              'font': {'size': 25}},
                    'number': {'valueformat': ',',
                              'font': {'size': 40}},
                    'domain': {'y': [0, 1], 'x': [0, 1]}}],
            'layout': go.Layout(
                font=dict(color='blue'),
                height=100
                )}
                      )
        ),
            ]
        )
    ],
    style={"width": "18rem" ,'margin' : '1rem' , 'text-align': 'center'},
),
                  dbc.Card(
    [  
        dbc.CardHeader(" Recovered  "),
        dbc.CardBody(
            [
                    html.Div(dcc.Graph(id='Recovered',
                       figure={'data': [{'type': 'indicator',
                    'mode': 'number+delta',
                    'value': Recovered_value,
                    'delta': {'reference': Recovered_delta,
                              'valueformat': ',g',
                              'relative': False,
                              'increasing': {'color': 'black'},
                              'decreasing': {'color': 'black'},
                              'font': {'size': 25}},
                    'number': {'valueformat': ',',
                              'font': {'size': 40}},
                    'domain': {'y': [0, 1], 'x': [0, 1]}}],
            'layout': go.Layout(
                font=dict(color='#33d3ab'),
                height=100
                )}
                      )
        ),
            ]
        )
    ],
    style={"width": "18rem",'margin' : '1rem', 'text-align': 'center'},
),
                  dbc.Card(
    [
        dbc.CardHeader(" Deaths to Date  "),
        dbc.CardBody(
            [
                
                
                    html.Div(dcc.Graph(id='deaths',
                       figure={'data': [{'type': 'indicator',
                    'mode': 'number+delta',
                    'value': Deaths_value,
                    'delta': {'reference': Deaths_delta,
                              'valueformat': ',g',
                              'relative': False,
                              'increasing': {'color': 'black'},
                              'decreasing': {'color': 'black'},
                              'font': {'size': 25}},
                    'number': {'valueformat': ',',
                              'font': {'size': 40}},
                    'domain': {'y': [0, 1], 'x': [0, 1]}}],
            'layout': go.Layout(
                font=dict(color='red'),
                height=100

                )}
                      )
        ),
            ]
        )
    ],
    style={"width": "18rem" ,'margin' : '1rem', 'text-align': 'center'}
)
            ]
        )])

In [27]:
graphs = html.Div(
    [
        dbc.Row([
            dbc.Col(
        dcc.Graph( 
                        id = 'top10CountryGraph',
                        figure = px.bar(countrysDF.head(10) , x = 'Country/Region', y = 'Confirmed' , title = 'Top 10 Reported Cases by Country',template = 'plotly_dark')
                    ),style = { 'margin' : '1rem'}),
            dbc.Col(
        dcc.Graph( 
                        id = 'MonthsGraph',
                        figure = px.line(top5Months , x = 'month', y = ['Confirmed','Deaths','Recovered','Active'],title = 'Monthly Reports From All over the World',template = 'plotly_dark')
                    ),style = { 'margin' : '1rem'})
        
        ])
    ])

In [28]:
interactiveGraphs = html.Div([
    dcc.Markdown('**Choose Country : ** ', style = {'color':'white' ,'font-size' : '40' , 'margin': '1rem'}),

    dcc.Dropdown(
            id = 'countryDropDown',
        options=[
            { 'label' : country , 'value' : country } for country in df['Country/Region'].unique()
        ],
        value='China'
    ),
    html.Br(),
                    dcc.Graph( 
                        id = 'CountryPermonthGraph',
                        figure = px.bar(x = x,y = y, title = 'Country Statistics Per Month' , color = x,labels=dict(x="Case", y="Number of Cases"),template = 'plotly_dark')
                    ),
    html.Br(),
    dcc.Slider(
        id = 'monthSlider',
        min=min(df.month),
        marks={str(i): '{} 2020'.format(calendar.month_abbr[i]) for i in df.month.unique()},
        value=min(df.month),
        max=max(df.month),
        step=None
    )
])


In [29]:
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = html.Div([row , graphs , interactiveGraphs],style = {'background-color' : 'black'})
@app.callback(
    Output('CountryPermonthGraph' , 'figure'),
    Input('countryDropDown','value'),
    Input('monthSlider' ,'value'))

def updateGraph(country , month):
    countryValues = collectedDF[(collectedDF['Country/Region'] == country) & (collectedDF.month == month)]
    countryValues = countryValues[['Confirmed','Deaths','Recovered','Active']]
    x = ['Confirmed','Deaths' , 'Recovered','Active']
    y = countryValues.values[0]
    return px.bar(x = x,y = y, title = 'Country Statistics Per Month', color = x,labels=dict(x="Case", y="Number of Cases"),template = 'plotly_dark')

##  Intializing dash app

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Oct/2021 12:49:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 12:49:29] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 12:49:29] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 12:49:29] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 12:49:29] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 12:49:29] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 12:49:29] "GET /_dash-component-suites/dash/dcc/async-markdown.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 12:49:29] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 12:49:29] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2021 12:49:29] "GET /_dash-component-suites/dash/dcc/async-highlight.js HTTP/1.1" 200 -


In [ ]:
# fig = px.scatter_mapbox(df,
#                         lat=df.Latitude,
#                         lon=df.Longitude,
#                         hover_name = df['Country/Region'],
#                         hover_data=['Confirmed','Deaths','Recovered' , 'population'],
#                         zoom=0)

# fig.update_layout(mapbox_style="open-street-map")
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()